<a href="https://colab.research.google.com/github/peremartra/llama-glu-expansion-pruning/blob/main/notebooks/03_Evaluate_1B_CARBON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GLU Pruning Research - Llama-3.2-1B Evaluation
## 03 - Measuring Environmental Impact & Inference Performance

### Exploring energy efficiency of width-pruned GLU models
by [Pere Martra](https://github.com/peremartra)

[![Paper](https://img.shields.io/badge/OSF-Paper-blue?logo=osf&logoColor=white)](https://doi.org/10.31219/osf.io/qgxea)
[![GitHub](https://img.shields.io/badge/⭐_Star-OptiPFair-orange?logo=github&logoColor=white)](https://github.com/peremartra/optipfair)
[![PyPI](https://img.shields.io/pypi/v/optipfair?logo=python&logoColor=white&label=v)](https://pypi.org/project/optipfair/)

**Repository:** [github.com/peremartra/llama-glu-expansion-pruning](https://github.com/peremartra/llama-glu-expansion-pruning)

---

**Colab Environment:** GPU L4 (or T4)

**Estimated Runtime:** ~1-2 hours total

---

## 📋 Notebook Objective

This notebook measures the inference performance and energy consumption of the Llama-3.2-1B model family (the base model and pruned versions from 10% to 60%). The goal is to quantify the efficiency gains (in speed and energy) achieved through structured pruning.

### Key Features:
✅ Inference Measurement: Measures Latency (Time To First Token or TTFT) and Throughput (tokens/sec).

✅ Energy Tracking: Uses codecarbon to measure net energy (kWh) and Joules per token (J/token), factoring out idle power consumption.

✅ Robust Measurements: Executes 3 "runs" with random seeds ([42, 123, 456]) for each model, ensuring reliable averages.

✅ On-the-fly Pruning: Loads the base model and applies pruning in memory (using optipfair) for each run, no pre-generated models needed.

✅ Checkpoint Support: Saves the results of each individual run, allowing resumption if the notebook is interrupted.

### Results will answer:
- How much energy (Joules/token) does pruning actually save?

- Does the 40% pruned model (the "star model" for accuracy) also have the best performance/energy profile?

- What is the trade-off between pruning percentage, latency (TTFT), and throughput?

- How does batch size (bsz=1 vs bsz=8) affect the efficiency of the pruned models?

---

**Note**: This notebook only measures inference performance and energy consumption. The accuracy evaluation (using lm-eval) is performed in the [02_Evaluate_1B.ipynb](https://github.com/peremartra/llama-glu-expansion-pruning/blob/main/notebooks/02_Evaluate_1B.ipynb) notebook.

---

# 1. Setup & Installation

In [ ]:
# Install required libraries
!pip install -q optipfair
!pip install -q lm-eval
!pip install -q langdetect
!pip install -q codecarbon

In [ ]:
# Mount Google Drive for checkpoint persistence
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Download utils.py from GitHub repository
!wget -q https://raw.githubusercontent.com/peremartra/llama-glu-expansion-pruning/main/utils.py

# Verify download
import os
if os.path.exists('utils.py'):
    print("✅ utils.py downloaded successfully")
else:
    print("❌ Failed to download utils.py")

In [ ]:
# Import core libraries and utilities
import torch
import json
import pandas as pd
from datetime import datetime
from pathlib import Path
import glob
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer

# Import our utility functions
from utils import (
    EXPERIMENT_CONFIG_CARBON,
    BENCHMARKS_CARBON,
    load_or_create_model,
    run_carbon_profiling,
    run_robust_evaluation,
    clear_gpu_cache,
    get_model_stats,
    format_results_table,
    calibrate_idle_power  # ← NUEVA FUNCIÓN
)

print("✅ All imports successful")
print(f"📱 Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# =============================================================================
# EXPERIMENTAL RUNS CONFIGURATION
# =============================================================================
NUM_EXPERIMENTAL_RUNS = 3
RANDOM_SEEDS = [42, 123, 456]  # Fixed seeds for reproducibility

print(f"🔄 Experiment Configuration:")
print(f"   Number of runs: {NUM_EXPERIMENTAL_RUNS}")
print(f"   Random seeds: {RANDOM_SEEDS}")
print(f"   Total evaluations per model: {len(BENCHMARKS_CARBON)} workloads × {NUM_EXPERIMENTAL_RUNS} runs")
print()

# 2. Configuration & Planning

This section filters the experiment configuration for 1B models and displays the profiling plan.

In [ ]:
# Filter configuration for 1B models only
models_1b = [
    config for config in EXPERIMENT_CONFIG_CARBON
    if "1B" in config["base_model"] and "3B" not in config["base_model"] and "Instruct" not in config["base_model"]
]

print(f"\n{'='*70}")
print("📊 EVALUATION PLAN: Llama-3.2-1B Family")
print(f"{'='*70}\n")

print(f"Total models to evaluate: {len(models_1b) + 1}")  # +1 for base model
print(f"Benchmarks per model: {len(BENCHMARKS_CARBON)}")
print(f"Total evaluations: {(len(models_1b) + 1) * len(BENCHMARKS_CARBON)}")
print(f"Estimated runtime: ~1-1.5 hours\n")

# Display models table
print("Models to profile:")
print("-" * 70)
print(f"{'Model':<40} {'Pruning':<10} {'Star':<6}")
print("-" * 70)

for config in models_1b:
    if config['pruning_pct'] == 0:
        model_name = "Llama-3.2-1B (baseline)"
    else:
        model_name = f"Llama-3.2-1B-pruned-{config['pruning_pct']}%"

    pruning = f"{config['pruning_pct']}%"
    star = "⭐ Yes" if config['is_star'] else "No"
    print(f"{model_name:<40} {pruning:<10} {star:<6}")

print("-" * 70)

# Display workloads
print("\nWorkloads to run:")
print("-" * 70)
for i, workload in enumerate(BENCHMARKS_CARBON, 1):
    name = workload['name']
    prompts = workload['num_prompts']
    tokens = workload['max_new_tokens']
    desc = workload['description']
    print(f"{i}. {name:<25} {prompts} prompts, {tokens} tokens - {desc}")
print("-" * 70)

print("\n⚙️ Configuration:")
print(f"   - Neuron selection method: MAW (Maximum Absolute Weight)")
print(f"   - Checkpointing: Enabled (per-workload granularity)")
print(f"   - Model creation: On-the-fly pruning (no pre-creation needed)")
print(f"   - GPU warm-up: First 5 prompts excluded from metrics")
print(f"   - Energy tracking: CodeCarbon")

In [ ]:
# Setup checkpoint paths (manual construction since helpers are internal)
BASE_CHECKPOINT_DIR = "/content/drive/MyDrive/glu_pruning/checkpoints"
RESULTS_DIR = "/content/drive/MyDrive/glu_pruning/results"

# Carbon-specific subdirectory
CHECKPOINT_DIR = f"{BASE_CHECKPOINT_DIR}/1b_carbon"

# Create directories if they don't exist
Path(CHECKPOINT_DIR).mkdir(parents=True, exist_ok=True)
Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)

print(f"✅ Checkpoint directory: {CHECKPOINT_DIR}")
print(f"✅ Results directory: {RESULTS_DIR}")

# ============================================================================
# CONSTRUCT CHECKPOINT PATHS DYNAMICALLY FROM models_1b
# ============================================================================
checkpoint_paths = {}

for config in models_1b:
    pruning_pct = config['pruning_pct']

    # Create key: "baseline" for 0%, "{X}pct" for others
    if pruning_pct == 0:
        key = "baseline"
        filename = "llama_3.2_1b_baseline_carbon.json"
    else:
        key = f"{pruning_pct}pct"
        filename = f"llama_3.2_1b_pruned_{pruning_pct}pct_carbon.json"

    checkpoint_paths[key] = f"{CHECKPOINT_DIR}/{filename}"

print(f"\n📂 Auto-generated {len(checkpoint_paths)} checkpoint paths:")
for key, path in checkpoint_paths.items():
    exists = "✅ Exists" if Path(path).exists() else "🆕 New"
    print(f"  {key:<10}: {exists}")

# 3. Baseline Evaluation

Evaluate the original Llama-3.2-1B model to establish performance baseline.

In [ ]:
# ====================================================================
# CALIBRATE IDLE POWER (Run once at start)
# ====================================================================
print("="*70)
print("🔋 STEP 0: IDLE POWER CALIBRATION")
print("="*70 + "\n")

idle_calibration = calibrate_idle_power(
    device="cuda",
    duration_seconds=30,  # 30s is enough for stable measurement
    verbose=True
)

# Save calibration to drive for reproducibility
calibration_path = f"{RESULTS_DIR}/idle_power_calibration.json"
with open(calibration_path, 'w') as f:
    json.dump(idle_calibration, f, indent=2)

print(f"\n💾 Calibration saved to: {calibration_path}")
print("="*70 + "\n")

In [ ]:
print(f"\n{'='*70}")
print("📊  PHASE 1: BASELINE PROFILING")
print(f"{'='*70}\n")

BASE_MODEL_ID = "meta-llama/Llama-3.2-1B"

# ============================================================================
# MULTIPLE RUNS LOOP FOR BASELINE
# ============================================================================
baseline_all_runs = {}  # Store results from all runs

for run_idx, seed in enumerate(RANDOM_SEEDS, 1):
    print(f"\n{''*70}")
    print(f"🔄  BASELINE - RUN {run_idx}/{NUM_EXPERIMENTAL_RUNS} (seed={seed})")
    print(f"{'─'*70}\n")

    # Load base model (fresh for each run to avoid state contamination)
    print(f"Loading base model: {BASE_MODEL_ID}...")
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_ID,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print("✅  Model loaded successfully")

    # Construct checkpoint path for this run
    checkpoint_path_run = checkpoint_paths["baseline"].replace(".json", f"_run{run_idx}_seed{seed}.json")

    # Run carbon profiling with seed
    baseline_results_run = run_carbon_profiling(
        model=base_model,
        tokenizer=tokenizer,
        workloads=BENCHMARKS_CARBON,
        checkpoint_path=checkpoint_path_run,
        model_name=f"Llama-3.2-1B-baseline-run{run_idx}",
        idle_power_calibration=idle_calibration,  # ← CAMBIO: usar None
        device="cuda",
        random_seed=seed  # ← NUEVO: pasar seed
    )

    # Store results for this run
    baseline_all_runs[f"run_{run_idx}"] = baseline_results_run

    # Display run summary
    print(f"\n✅ Run {run_idx} completed")
    print("Results summary:")
    for workload_name, metrics in baseline_results_run.items():
        print(f"\n{workload_name}:")
        print(f"  Energy: {metrics['energy_kwh']:.6f} kWh")
        print(f"  Throughput: {metrics['throughput_tokens_per_sec']:.2f} tok/s")

    # Clear memory before next run
    del base_model
    clear_gpu_cache()
    print(f"🧹  Memory cleared\n")

print(f"\n{'='*70}")
print("✅  ALL BASELINE RUNS COMPLETED")
print(f"{'='*70}\n")

# 4. Pruned Models Evaluation Loop

Profile the pruned variants using on-the-fly pruning with OptiPFair.

In [ ]:
print(f"\n{'='*70}")
print("📊  PHASE 2: PRUNED MODELS PROFILING")
print(f"{'='*70}\n")

# Store all results for final comparison
# Structure: all_results[model_key][run_key] = results_dict
all_results = {
    "baseline": baseline_all_runs  # ← CAMBIO: ahora contiene múltiples runs
}

# Filter out baseline (already done)
pruned_models = [m for m in models_1b if m['pruning_pct'] > 0]

# Evaluate each pruned model
for i, config in enumerate(pruned_models, 1):
    model_name = f"Llama-3.2-1B-pruned-{config['pruning_pct']}%"
    pruning_pct = config['pruning_pct']
    is_star = config['is_star']

    print(f"\n{'─'*70}")
    print(f"🔄  PROFILING MODEL {i}/{len(pruned_models)}: {model_name}")
    print(f"   Pruning: {pruning_pct}% | Star: {'⭐ ' if is_star else '❌'}")
    print(f"{'─'*70}\n")

    # ========================================================================
    # MULTIPLE RUNS LOOP FOR THIS PRUNED MODEL
    # ========================================================================
    pruned_model_all_runs = {}

    for run_idx, seed in enumerate(RANDOM_SEEDS, 1):
        print(f"\n{'··'*35}")
        print(f"   RUN {run_idx}/{NUM_EXPERIMENTAL_RUNS} (seed={seed})")
        print(f"{'··'*35}\n")

        try:
            # Load or create model (fresh for each run)
            model, tokenizer, stats = load_or_create_model(config, device="cuda")

            # Display model statistics (once per model, not per run)
            if run_idx == 1:
                print(f"\n📈  Model Statistics:")
                print(f"   Parameters: {stats['total_parameters']:,}")
                print(f"   Size: {stats['size_gb']:.2f} GB")
                if 'pruning_stats' in stats:
                    print(f"   Reduction: {stats['pruning_stats']['percentage_reduction']:.1f}%")
                print(f"   Source: {stats['source']}\n")

            # Construct checkpoint path for this run
            checkpoint_key = f"{pruning_pct}pct"
            checkpoint_path_run = checkpoint_paths[checkpoint_key].replace(".json", f"_run{run_idx}_seed{seed}.json")

            # Run profiling with seed
            results_run = run_carbon_profiling(
                model=model,
                tokenizer=tokenizer,
                workloads=BENCHMARKS_CARBON,
                checkpoint_path=checkpoint_path_run,
                model_name=f"{model_name}-run{run_idx}",
                idle_power_calibration=idle_calibration,  # ← CAMBIO: usar None
                device="cuda",
                random_seed=seed  # ← NUEVO: pasar seed
            )

            # Store results for this run
            pruned_model_all_runs[f"run_{run_idx}"] = results_run

            print(f"\n✅ Run {run_idx} completed for {model_name}")

            # Clear memory before next run
            del model
            clear_gpu_cache()

        except Exception as e:
            print(f"\n❌  ERROR in run {run_idx}: {str(e)}")
            import traceback
            traceback.print_exc()
            clear_gpu_cache()
            continue

    # Store all runs for this model
    all_results[checkpoint_key] = pruned_model_all_runs

    print(f"\n✅ All runs completed for {model_name}\n")

print(f"\n{'='*70}")
print("✅  ALL MODELS PROFILED")
print(f"{'='*70}\n")

In [ ]:
# ===========================================================================
# CREATE GLOBAL MAPPINGS (FOR LATER CELLS)
# ===========================================================================
# These dicts are used by cells 17, 18, and 21 for aggregation and display.

model_names = {}
model_pruning = {}
model_is_star = {}

print("\n🛠️  Creating global model mappings (model_names, model_pruning, model_is_star)...")

for config in models_1b:
    pruning_pct = config['pruning_pct']

    # Determine the key (must match Cell 10's logic: 'baseline' or '40pct')
    if pruning_pct == 0:
        key = "baseline"
        model_name = "Llama-3.2-1B (baseline)"
    else:
        key = f"{pruning_pct}pct"
        model_name = f"Llama-3.2-1B-pruned-{pruning_pct}%"

    # Populate the dictionaries
    model_names[key] = model_name
    model_pruning[key] = pruning_pct
    model_is_star[key] = config['is_star']

print(f"   ...Mappings created for {len(model_names)} models: {list(model_names.keys())}")

# 5. Results Consolidation & Export

Consolidate all evaluation results and export to CSV for analysis.

In [ ]:
print(f"\n{'='*70}")
print("📊  AGGREGATING RESULTS ACROSS RUNS")
print(f"{'='*70}\n")

JOULES_THRESHOLD = 0.30  # 300 mJ/token
OUTLIER_RUN_THRESHOLD = 0.3  # Si >80% de benchmarks son outliers, descartar run completo

def detect_problematic_runs(runs_dict):
    """
    Detecta runs donde la mayoría de benchmarks son outliers.
    Retorna un dict con el % de outliers por run.
    """
    run_outlier_stats = {}

    for run_key, workloads in runs_dict.items():
        outlier_count = 0
        total_count = len(workloads)

        for workload_name, data in workloads.items():
            joules = data.get('joules_per_token', 0.0)
            if joules > JOULES_THRESHOLD:
                outlier_count += 1

        outlier_percentage = outlier_count / total_count if total_count > 0 else 0
        run_outlier_stats[run_key] = {
            'outlier_count': outlier_count,
            'total_count': total_count,
            'outlier_percentage': outlier_percentage,
            'is_problematic': outlier_percentage > OUTLIER_RUN_THRESHOLD
        }

    return run_outlier_stats

def aggregate_runs(runs_dict):
    """
    Aggregate metrics from multiple runs with intelligent outlier handling.
    """
    import numpy as np

    # PASO 1: Detectar runs completamente problemáticos
    run_stats = detect_problematic_runs(runs_dict)

    print("\n    🔍 Run quality assessment:")
    for run_key, stats in run_stats.items():
        status = "❌ PROBLEMATIC" if stats['is_problematic'] else "✅ OK"
        print(f"    {run_key}: {stats['outlier_count']}/{stats['total_count']} outliers ({stats['outlier_percentage']*100:.0f}%) - {status}")

    # Filtrar runs problemáticos
    valid_runs = {k: v for k, v in runs_dict.items()
                  if not run_stats[k]['is_problematic']}
    problematic_runs = {k: v for k, v in runs_dict.items()
                       if run_stats[k]['is_problematic']}

    if not valid_runs:
        print("\n    ⚠️ WARNING: All runs are problematic. Using all runs with per-benchmark filtering.")
        valid_runs = runs_dict  # Usar todos si no hay ninguno válido
    else:
        print(f"\n    ✅ Using {len(valid_runs)}/{len(runs_dict)} runs for aggregation")
        if problematic_runs:
            print(f"    🚫 Excluding {len(problematic_runs)} problematic runs: {list(problematic_runs.keys())}")

    # PASO 2: Agregar por workload
    aggregated = {}
    outliers_log = {}

    first_valid_run_key = next((key for key, val in valid_runs.items() if val), None)
    if not first_valid_run_key:
        print("    ⚠️ Error: All runs dictionaries are empty.")
        return {}, {}, problematic_runs

    workload_names = list(valid_runs[first_valid_run_key].keys())

    for workload_name in workload_names:
        energy_samples = []
        throughput_samples = []
        ttft_samples = []
        joules_per_token_samples = []

        print(f"\n    Processing {workload_name}...")
        run_data = {}
        valid_seeds_used = []
        outliers_detected = []

        for run_key in valid_runs.keys():
            if workload_name not in valid_runs[run_key]:
                print(f"      ⚠️ Warning: '{workload_name}' missing in '{run_key}'")
                continue

            run_data_current = valid_runs[run_key][workload_name]
            joules = run_data_current.get('joules_per_token', 0.0)

            # Filtrado por benchmark
            if joules > JOULES_THRESHOLD:
                outlier_info = {
                    "run_key": run_key,
                    "joules_per_token": source_data.get('joules_per_token', 0.0),
                    "energy_kwh": run_data_current['energy_kwh'],
                    "reason": f">{JOULES_THRESHOLD:.4f} J/token"
                }
                outliers_detected.append(outlier_info)
                print(f"      🚫 Outlier: {run_key} = {joules:.4f} J/token")
            else:
                # Valor válido
                energy_samples.append(run_data_current['energy_kwh'])
                throughput_samples.append(run_data_current['throughput_tokens_per_sec'])
                joules_per_token_samples.append(joules)

                if run_data_current.get('avg_ttft_ms') is not None:
                    ttft_samples.append(run_data_current['avg_ttft_ms'])

                try:
                    run_index = int(run_key.split('_')[-1]) - 1
                    if 0 <= run_index < len(RANDOM_SEEDS):
                        valid_seeds_used.append(RANDOM_SEEDS[run_index])
                except ValueError:
                    print(f"      ⚠️ Could not parse seed from '{run_key}'")

                run_data = run_data_current

        if outliers_detected:
            outliers_log[workload_name] = outliers_detected

        # Verificar datos válidos
        if not energy_samples:
            print(f"      ❌ No valid data for {workload_name}")
            aggregated[workload_name] = {
                "error": "All measurements were outliers",
                "outliers_excluded": outliers_detected
            }
            continue

        print(f"      ✅ {len(energy_samples)} valid measurements")

        # Raw runs solo con válidos
        valid_raw_runs = {}
        for run_key in valid_runs.keys():
            if workload_name in valid_runs[run_key]:
                source_data = valid_runs[run_key][workload_name]

                valid_raw_runs[run_key] = {
                    # --- Datos existentes ---
                    "energy_kwh": source_data['energy_kwh'], # Net (actual)
                    "throughput": source_data['throughput_tokens_per_sec'],
                    "ttft_ms": source_data.get('avg_ttft_ms'),
                    "joules_per_token": source_data.get('joules_per_token', 0.0),

                    # --- NUEVOS DATOS AÑADIDOS ---
                    "energy_raw_kwh": source_data.get('energy_raw_kwh'),
                    "energy_idle_kwh": source_data.get('energy_idle_kwh'),
                    "total_new_tokens": source_data.get('total_new_tokens'),
                    "inference_duration_sec": source_data.get('hardware_metadata', {}).get('duration_sec',
                          source_data.get('total_loop_time_sec')),
                    "memory_allocated_gb": source_data.get('model_size_gb') # O 'memory_allocated_gb' si está disponible
                }

        # Calcular estadísticas
        aggregated[workload_name] = {
            "energy_kwh_mean": float(np.mean(energy_samples)),
            "energy_kwh_std": float(np.std(energy_samples, ddof=1)) if len(energy_samples) > 1 else 0.0,
            "energy_kwh_min": float(np.min(energy_samples)),
            "energy_kwh_max": float(np.max(energy_samples)),

            "throughput_mean": float(np.mean(throughput_samples)),
            "throughput_std": float(np.std(throughput_samples, ddof=1)) if len(throughput_samples) > 1 else 0.0,
            "throughput_min": float(np.min(throughput_samples)),
            "throughput_max": float(np.max(throughput_samples)),

            "ttft_mean": float(np.mean(ttft_samples)) if ttft_samples else None,
            "ttft_std": float(np.std(ttft_samples, ddof=1)) if len(ttft_samples) > 1 else None,
            "ttft_min": float(np.min(ttft_samples)) if ttft_samples else None,
            "ttft_max": float(np.max(ttft_samples)) if ttft_samples else None,

            "joules_per_token_mean": float(np.mean(joules_per_token_samples)),
            "joules_per_token_std": float(np.std(joules_per_token_samples, ddof=1)) if len(joules_per_token_samples) > 1 else 0.0,
            "joules_per_token_min": float(np.min(joules_per_token_samples)),
            "joules_per_token_max": float(np.max(joules_per_token_samples)),

            "num_runs_aggregated": len(energy_samples),
            "seeds_used": valid_seeds_used,
            "model_size_gb": run_data.get('model_size_gb', 0),
            "batch_size": run_data.get('batch_size', 0),
            "num_prompts": run_data.get('num_prompts', 0),

            "outliers_excluded": outliers_detected if outliers_detected else None,
            "outliers_count": len(outliers_detected),

            "raw_runs_data": valid_raw_runs
        }

    return aggregated, outliers_log, problematic_runs

# Aggregate results
aggregated_results = {}
all_outliers_log = {}
all_problematic_runs = {}

if 'all_results' not in globals():
    print("❌ ERROR: 'all_results' not defined.")
else:
    for model_key, runs_dict in all_results.items():
        print(f"\n{'='*70}")
        print(f"Aggregating {model_key}...")
        print(f"{'='*70}")

        aggregated, outliers_log, problematic_runs = aggregate_runs(runs_dict)
        aggregated_results[model_key] = aggregated

        if outliers_log:
            all_outliers_log[model_key] = outliers_log
        if problematic_runs:
            all_problematic_runs[model_key] = problematic_runs

    print(f"\n{'='*70}")
    print("✅ AGGREGATION COMPLETE")
    print(f"{'='*70}")
    print(f"   Models processed: {len(aggregated_results)}")
    print(f"   Total problematic runs excluded: {sum(len(v) for v in all_problematic_runs.values())}")
    print(f"   Total individual outliers: {sum(len(v) for v in all_outliers_log.values())}")

    # Reporte detallado
    if all_problematic_runs:
        print(f"\n{'='*70}")
        print("🚫 PROBLEMATIC RUNS EXCLUDED (>50% outliers)")
        print(f"{'='*70}")
        for model_key, runs in all_problematic_runs.items():
            print(f"\n{model_key}:")
            for run_key in runs.keys():
                print(f"  ❌ {run_key}")

    if all_outliers_log:
        print(f"\n{'='*70}")
        print("🚫 INDIVIDUAL OUTLIERS EXCLUDED")
        print(f"{'='*70}")
        for model_key, workloads in all_outliers_log.items():
            print(f"\n{model_key}:")
            for workload_name, outliers in workloads.items():
                print(f"  {workload_name}:")
                for outlier in outliers:
                    print(f"    - {outlier['run_key']}: {outlier['joules_per_token']:.4f} J/token")

print("\n" + "="*70 + "\n")

In [ ]:
print(f"\n{'='*70}")
print("📊  CONSOLIDATING AGGREGATED RESULTS TO CSV/JSON")
print(f"{'='*70}\n")

# Build consolidated data using AGGREGATED results
consolidated_data = []

for model_key, workload_results in aggregated_results.items():
    display_name = model_names.get(model_key, model_key)
    pruning_pct = model_pruning.get(model_key, 0)
    is_star = model_is_star.get(model_key, False)

    for workload_name, metrics in workload_results.items():
        row = {
            "model": display_name,
            "pruning_pct": pruning_pct,
            "is_star": is_star,
            "workload": workload_name,

            # Aggregated metrics (mean ± std)
            "energy_kwh_mean": metrics["energy_kwh_mean"],
            "energy_kwh_std": metrics["energy_kwh_std"],
            "throughput_mean": metrics["throughput_mean"],
            "throughput_std": metrics["throughput_std"],
            "joules_per_token_mean": metrics["joules_per_token_mean"],
            "joules_per_token_std": metrics["joules_per_token_std"],
            "ttft_mean": metrics.get("ttft_mean"),
            "ttft_std": metrics.get("ttft_std"),

            # Metadata
            "num_runs": metrics["num_runs_aggregated"], # <--- LÍNEA CORREGIDA
            "model_size_gb": metrics["model_size_gb"],
            "batch_size": metrics["batch_size"],
        }
        consolidated_data.append(row)

# Create DataFrame
df = pd.DataFrame(consolidated_data)
df = df.sort_values(by=["pruning_pct", "workload"]).reset_index(drop=True)

print(f"✅  Consolidated {len(df)} result rows (with aggregated statistics)")
print(f"   Models: {df['model'].nunique()}")
print(f"   Workloads: {df['workload'].nunique()}")
# Usamos el dataframe para asegurar que contamos los runs correctos
print(f"   Runs per configuration: {df['num_runs'].max()}")

# Display preview with mean ± std
print("\nDataFrame Preview (Aggregated Results):")
print(df[['model', 'workload', 'energy_kwh_mean', 'energy_kwh_std', 'throughput_mean', 'throughput_std']].head(10))

# Save to CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_path = f"{RESULTS_DIR}/carbon_1b_aggregated_results_{timestamp}.csv"
df.to_csv(csv_path, index=False)

print(f"\n💾  Aggregated results saved to: {csv_path}")

# Also save raw runs for transparency
raw_runs_path = f"{RESULTS_DIR}/carbon_1b_raw_runs_{timestamp}.json"
with open(raw_runs_path, 'w') as f:
    json.dump(all_results, f, indent=2, default=str)  # default=str handles numpy types

print(f"💾  Raw runs data saved to: {raw_runs_path}")

In [ ]:
print(f"\n{'='*70}")
print("📈 CARBON & PERFORMANCE ANALYSIS")
print(f"S{'='*70}\n")

# Calculate averages per model across workloads
summary_data = []

# ===================================================================
# START: FIX
# ===================================================================

# Use dynamic keys from aggregated_results (FIX 1)
for model_key in aggregated_results.keys():
    # Get the aggregated workload results for this model (FIX 2)
    workload_results = aggregated_results[model_key]

    # Aggregate metrics across workloads using the '_mean' keys (FIX 3)
    total_energy = sum(m['energy_kwh_mean'] for m in workload_results.values())
    avg_throughput = np.mean([m['throughput_mean'] for m in workload_results.values()])
    ttft_values = [m['ttft_mean'] for m in workload_results.values() if m.get('ttft_mean') is not None]

    # --- AÑADIDO ---
    joules_per_token_values = [m['joules_per_token_mean'] for m in workload_results.values() if m.get('joules_per_token_mean') is not None]
    avg_joules_per_token = np.mean(joules_per_token_values) if joules_per_token_values else 0.0
    # --- FIN AÑADIDO ---
# ===================================================================
# END: FIX
# ===================================================================

    avg_ttft = np.mean(ttft_values) if ttft_values else 0.0 # Usar np.mean
    model_size = list(workload_results.values())[0]['model_size_gb']  # Same for all workloads

    summary = {
        "model": model_names.get(model_key, model_key),
        "pruning_pct": model_pruning.get(model_key, 0),
        "is_star": model_is_star.get(model_key, False),
        "total_energy_kwh": total_energy,
        "avg_throughput_tok_s": avg_throughput,
        "avg_joules_per_token": avg_joules_per_token, # <-- AÑADIDO
        "avg_ttft_ms": avg_ttft,
        "model_size_gb": model_size
    }
    summary_data.append(summary)

# Sort by pruning_pct for consistent display
summary_df = pd.DataFrame(summary_data).sort_values('pruning_pct').reset_index(drop=True)

# --- ACTUALIZAR PRINT ---
print("Performance Summary (Calculations Corrected):")
print("-" * 110) # Ampliar tabla
print(summary_df.to_string(index=False, float_format="%.4f"))
print("-" * 110) # Ampliar tabla
# --- FIN ACTUALIZAR PRINT ---

# Calculate improvements vs baseline
baseline_row = summary_df[summary_df['pruning_pct'] == 0].iloc[0]

print(f"\n💡 Performance vs. Baseline:")
print("-" * 90)

for _, row in summary_df.iterrows():
    if row['pruning_pct'] == 0:
        continue

    # === METRICS WHERE LESS IS BETTER (calculate REDUCTION) ===
    # Positive = improvement (less consumption/latency)
    # Negative = degradation (more consumption/latency)

    energy_total_reduction = ((baseline_row['total_energy_kwh'] - row['total_energy_kwh']) / baseline_row['total_energy_kwh']) * 100

    joules_per_token_reduction = ((baseline_row['avg_joules_per_token'] - row['avg_joules_per_token']) / baseline_row['avg_joules_per_token']) * 100

    ttft_reduction = ((baseline_row['avg_ttft_ms'] - row['avg_ttft_ms']) / baseline_row['avg_ttft_ms']) * 100

    size_reduction = ((baseline_row['model_size_gb'] - row['model_size_gb']) / baseline_row['model_size_gb']) * 100

    # === METRICS WHERE MORE IS BETTER (calculate INCREASE) ===
    # Positive = improvement (more throughput/tokens)
    # Negative = degradation (less throughput/tokens)

    throughput_increase = ((row['avg_throughput_tok_s'] - baseline_row['avg_throughput_tok_s']) / baseline_row['avg_throughput_tok_s']) * 100

    tokens_increase = ((row['total_generated_tokens'] - baseline_row['total_generated_tokens']) / baseline_row['total_generated_tokens']) * 100

    # Display results
    star_marker = "⭐" if row['is_star'] else ""
    print(f"\n{row['model']} {star_marker}")

    # Energy (Total) - LESS IS BETTER
    print(f"  Energy (Total): {energy_total_reduction:+.2f}% ({'✅ Lower' if energy_total_reduction > 0 else '❌ Higher'})")

    # Tokens Generated - MORE IS BETTER
    print(f"  Tokens Generated: {tokens_increase:+.2f}% ({'✅ More Output' if tokens_increase > 0 else '⚠️ Less Output'})")

    # Energy per Token - LESS IS BETTER
    print(f"  Energy (J/Token): {joules_per_token_reduction:+.2f}% ({'✅ More Efficient' if joules_per_token_reduction > 0 else '❌ Less Efficient'})")

    # Throughput - MORE IS BETTER
    print(f"  Throughput: {throughput_increase:+.2f}% ({'✅ Faster' if throughput_increase > 0 else '⚠️ Slower'})")

    # Latency (TTFT) - LESS IS BETTER
    print(f"  Latency (TTFT): {ttft_reduction:+.2f}% ({'✅ Lower' if ttft_reduction > 0 else '⚠️ Higher'})")

    # Model Size - LESS IS BETTER
    print(f"  Model Size: {size_reduction:+.2f}% ({'✅ Smaller' if size_reduction > 0 else '⚠️ Larger'})")

print("\n" + "-" * 90)

# 6. Quick Analysis & Visualization

Generate quick insights to decide which models merit uploading to HuggingFace Hub.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

models = summary_df['model'].values
pruning_levels = summary_df['pruning_pct'].values
star_mask = summary_df['is_star'].values

# Colors: baseline blue, star gold, others gray
colors = ['#1f77b4' if p == 0 else '#FFD700' if s else '#95a5a6'
          for p, s in zip(pruning_levels, star_mask)]

# 1. Energy Consumption
axes[0, 0].bar(range(len(models)), summary_df['total_energy_kwh'], color=colors, edgecolor='black', linewidth=1.5)
axes[0, 0].set_xticks(range(len(models)))
axes[0, 0].set_xticklabels([m.replace('Llama-3.2-', '') for m in models], rotation=0, ha='center')
axes[0, 0].set_ylabel('Total Energy (kWh)', fontsize=11)
axes[0, 0].set_title('Energy Consumption', fontsize=13, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(summary_df['total_energy_kwh']):
    axes[0, 0].text(i, v + 0.0001, f'{v:.4f}', ha='center', va='bottom', fontsize=9)

# 2. Throughput
axes[0, 1].bar(range(len(models)), summary_df['avg_throughput_tok_s'], color=colors, edgecolor='black', linewidth=1.5)
axes[0, 1].set_xticks(range(len(models)))
axes[0, 1].set_xticklabels([m.replace('Llama-3.2-', '') for m in models], rotation=0, ha='center')
axes[0, 1].set_ylabel('Throughput (tokens/s)', fontsize=11)
axes[0, 1].set_title('Inference Throughput', fontsize=13, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(summary_df['avg_throughput_tok_s']):
    axes[0, 1].text(i, v + 0.5, f'{v:.1f}', ha='center', va='bottom', fontsize=9)

# 3. Latency (TTFT)
axes[1, 0].bar(range(len(models)), summary_df['avg_ttft_ms'], color=colors, edgecolor='black', linewidth=1.5)
axes[1, 0].set_xticks(range(len(models)))
axes[1, 0].set_xticklabels([m.replace('Llama-3.2-', '') for m in models], rotation=0, ha='center')
axes[1, 0].set_ylabel('Avg TTFT (ms)', fontsize=11)
axes[1, 0].set_title('Latency (Time To First Token)', fontsize=13, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(summary_df['avg_ttft_ms']):
    axes[1, 0].text(i, v + 1, f'{v:.1f}', ha='center', va='bottom', fontsize=9)

# 4. Model Size
axes[1, 1].bar(range(len(models)), summary_df['model_size_gb'], color=colors, edgecolor='black', linewidth=1.5)
axes[1, 1].set_xticks(range(len(models)))
axes[1, 1].set_xticklabels([m.replace('Llama-3.2-', '') for m in models], rotation=0, ha='center')
axes[1, 1].set_ylabel('Model Size (GB)', fontsize=11)
axes[1, 1].set_title('Model Size', fontsize=13, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(summary_df['model_size_gb']):
    axes[1, 1].text(i, v + 0.02, f'{v:.2f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig(f"{RESULTS_DIR}/carbon_1b_analysis.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Visualization saved to: {RESULTS_DIR}/carbon_1b_analysis.png")

# 7. Agregating results.

In [ ]:
# NUEVO CÓDIGO PARA LA CELDA 42

import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path

print(f"\n{'='*70}")
print("📊 CONSTRUCTING FINAL JSON FROM AGGREGATED RESULTS")
print(f"{'='*70}\n")

# --- Validar variables globales ---
# (Asegurarse de que las variables de celdas anteriores existen)

if 'RESULTS_DIR' not in globals():
    print("⚠️ Warning: RESULTS_DIR not set. Using default './results'")
    RESULTS_DIR = "/content/drive/MyDrive/glu_pruning/results"

if 'BENCHMARKS_CARBON' not in globals():
    print("❌ Error: BENCHMARKS_CARBON not defined. Recargando fallback.")
    BENCHMARKS_CARBON = [
        {"name": "gsm8k_latency_b1", "num_prompts": 10, "max_new_tokens": 100, "dataset": "gsm8k", "subset": "test", "description": "Math reasoning (Latency, TTFT, bsz=1)", "batch_size": 1},
        {"name": "mmlu_latency_b1", "num_prompts": 10, "max_new_tokens": 50, "dataset": "mmlu", "subset": "test", "description": "Knowledge QA (Latency, TTFT, bsz=1)", "batch_size": 1},
        {"name": "ifeval_latency_b1", "num_prompts": 10, "max_new_tokens": 150, "dataset": "IFEval", "subset": "train", "description": "Instruction (Latency, TTFT, bsz=1)", "batch_size": 1},
        {"name": "gsm8k_throughput_b8", "num_prompts": 10, "max_new_tokens": 100, "dataset": "gsm8k", "subset": "test", "description": "Math reasoning (Throughput, bsz=8)", "batch_size": 8},
        {"name": "mmlu_throughput_b8", "num_prompts": 10, "max_new_tokens": 50, "dataset": "mmlu", "subset": "test", "description": "Knowledge QA (Throughput, bsz=8)", "batch_size": 8},
        {"name": "ifeval_throughput_b8", "num_prompts": 10, "max_new_tokens": 150, "dataset": "IFEval", "subset": "train", "description": "Instruction (Throughput, bsz=8)", "batch_size": 8},
    ]

# --- Usar resultados ya agregados y filtrados ---
if 'aggregated_results' not in globals() or 'all_results' not in globals():
    print("❌ FATAL ERROR: 'aggregated_results' o 'all_results' no están definidos.")
    print("   Por favor, ejecute de nuevo las celdas 38 y 39 antes de esta.")
else:
    print(f"Usando 'aggregated_results' (calculados en Celda 39) para {len(aggregated_results)} modelos.")

    # --- Construir el JSON Completo ---

    # 1. Construir mapeo de configuración
    config_map = {}
    model_family_name = "Llama-3.2-1B"

    for cfg in EXPERIMENT_CONFIG_CARBON:
        if model_family_name not in cfg["base_model"]:
            continue

        pruning_pct = cfg['pruning_pct']
        # La clave debe coincidir con la usada en 'aggregated_results' (p.ej. 'baseline', '20pct', '40pct')
        key = "baseline" if pruning_pct == 0 else f"{pruning_pct}pct"
        model_name_cfg = f"{model_family_name}" if pruning_pct == 0 else f"{model_family_name}-pruned-{pruning_pct}%"

        hf_repo = cfg.get("hf_repo_id", cfg["base_model"]) if pruning_pct > 0 else cfg["base_model"]

        config_map[key] = {
            "name": model_name_cfg,
            "pruning_pct": int(pruning_pct),
            "is_star": bool(cfg["is_star"]),
            "hf_repo": hf_repo,
        }

    print(f"\nDynamically built config map for {len(config_map)} models.")

    # 2. Construir 'models_evaluated'
    models_evaluated = {}
    summary_stats_list = []

    for model_key, workload_results in aggregated_results.items():
        if model_key in config_map:
            config = config_map[model_key]

            # Usar los resultados de workload_results (ya filtrados por outliers)
            models_evaluated[model_key] = {
                "name": config["name"],
                "pruning_pct": config["pruning_pct"],
                "is_star": config["is_star"],
                "hf_repo": config["hf_repo"],
                "results": workload_results
            }

            # Calcular estadísticas de resumen (basado en Celda 40)
            valid_workloads = [w for w in workload_results.values() if 'error' not in w]
            if valid_workloads:
                joules_per_token_values = [m['joules_per_token_mean'] for m in valid_workloads if m.get('joules_per_token_mean') is not None]
                avg_joules_per_token = float(np.mean(joules_per_token_values)) if joules_per_token_values else 0.0
                ttft_values = [m['ttft_mean'] for m in valid_workloads if m.get('ttft_mean') is not None]
                avg_ttft = float(np.mean(ttft_values)) if ttft_values else 0.0

                summary_stats_list.append({
                    "model": config["name"],
                    "pruning_pct": config["pruning_pct"],
                    "is_star": config["is_star"],
                    "total_energy_kwh": sum(m.get('energy_kwh_mean', 0) for m in valid_workloads),
                    "avg_throughput_tok_s": np.mean([m.get('throughput_mean', 0) for m in valid_workloads]),
                    "avg_joules_per_token": avg_joules_per_token,
                    "avg_ttft_ms": avg_ttft,
                    "model_size_gb": list(valid_workloads)[0].get('model_size_gb', 0)
                })
        else:
            print(f"⚠️ Warning: No config map entry found for key '{model_key}'.")

    # 3. Construir 'summary_statistics'
    baseline_stats = next((s for s in summary_stats_list if s['pruning_pct'] == 0), None)
    pruned_stats = sorted([s for s in summary_stats_list if s['pruning_pct'] > 0], key=lambda x: x['pruning_pct'])

    summary_statistics = {
        "baseline": baseline_stats,
        "pruned_models": pruned_stats
    }

    # 4. Obtener hardware_info (del primer run del baseline, usando 'all_results')
    hardware_info = {}
    try:
        # 1. CORRECCIÓN: Usar la clave correcta 'run_1' (generada en la Celda 9)
        run_key_to_check = "run_1"

        if "baseline" in all_results and run_key_to_check in all_results["baseline"]:
            # 2. Obtener la metadata del primer run/workload como fuente
            first_workload_data = list(all_results["baseline"][run_key_to_check].values())[0]

            if "hardware_metadata" in first_workload_data:
                hw_meta = first_workload_data["hardware_metadata"]

                # 3. CONSTRUCCIÓN CURADA: Crear el diccionario limpio que especificaste
                hardware_info = {
                    "gpu_model": hw_meta.get("gpu_name_torch"),
                    "gpu_memory_gb": round(hw_meta.get("gpu_total_memory_gb", 0), 2),
                    "gpu_compute_capability": hw_meta.get("gpu_compute_capability"),
                    "cuda_version": hw_meta.get("cuda_version"),
                    "torch_version": hw_meta.get("torch_version"),
                    "gpu_temperature_celsius": hw_meta.get("gpu_temperature_celsius"),
                }

                # 4. AÑADIR DATOS DE IDLE: Obtener de la variable global 'idle_calibration' (de la Celda 8)
                if 'idle_calibration' in globals():
                    hardware_info["idle_power_watts"] = round(idle_calibration.get("idle_power_watts", 0), 2)
                    hardware_info["idle_power_measurement_duration_sec"] = idle_calibration.get("duration_seconds")
                    hardware_info["idle_power_measurement_timestamp"] = idle_calibration.get("timestamp")
                else:
                    print("    Warning: 'idle_calibration' no encontrada. Faltarán datos de idle power.")

                print(f"    Successfully captured and curated hardware metadata (GPU: {hardware_info.get('gpu_model', 'N/A')})")
            else:
                print("    'hardware_metadata' not found in first workload.")
        else:
            print(f"    Could not find '{run_key_to_check}' in 'all_results[\"baseline\"]'. Keys available: {list(all_results.get('baseline', {}).keys())}")

    except Exception as e:
        print(f"    Error extracting hardware info: {e}")
        import traceback
        traceback.print_exc()


    # 5. Consolidar todo
    complete_results = {
        "experiment_metadata": {
            "timestamp": datetime.now().isoformat(),
            "notebook": "03_Evaluate_1B_CARBON.ipynb",
            "model_family": model_family_name,
            "pruning_method": "MAW (Maximum Absolute Weight)",
            "hardware_details": hardware_info
        },
        "benchmarks": [
            {
                "name": task["name"],
                "num_prompts": task["num_prompts"],
                "max_new_tokens": task["max_new_tokens"],
                "description": task["description"]
            }
            for task in BENCHMARKS_CARBON
        ],
        "models_evaluated": models_evaluated,
        "summary_statistics": summary_statistics,
        "citation": {
            "paper": "Exploring GLU Expansion Ratios: Structured Pruning in Llama-3.2 Models",
            "author": "Pere Martra",
            "doi": "https://doi.org/10.31219/osf.io/qgxea",
            "github": "https://github.com/peremartra/llama-glu-expansion-pruning",
            "note": "Results are freely available for research purposes. Please cite the paper if you use this data."
        }
    }
    print(f"\nSuccessfully consolidated results for {len(models_evaluated)} models.")

    # --- Guardar en JSON ---
    try:
        os.makedirs(RESULTS_DIR, exist_ok=True)

        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        json_path = f"{RESULTS_DIR}/llama_1b_carbon_complete_results_{timestamp}.json"
        with open(json_path, 'w') as f:
            json.dump(complete_results, f, indent=2, ensure_ascii=False, default=str)

        print(f"\n✅ Complete (and filtered) carbon results saved to:")
        print(f"   {json_path}")

        latest_json = f"{RESULTS_DIR}/llama_1b_carbon_complete_results_latest.json"
        with open(latest_json, 'w') as f:
            json.dump(complete_results, f, indent=2, ensure_ascii=False, default=str)

        print(f"✅ Latest (and filtered) version:")
        print(f"   {latest_json}")

        file_size_kb = Path(json_path).stat().st_size / 1024
        print(f"\n📊 File size: {file_size_kb:.1f} KB")

    except Exception as e:
        print(f"❌ Error saving JSON files: {e}")

    print(f"\n{'='*70}")
    print("✅ COMPLETE CARBON RESULTS SAVED - Ready for research sharing")
    print(f"{'='*70}\n")

---

## 🎓 Key Takeaways

This notebook evaluated the Llama-3.2-1B model family across different datasets to determine:

1. **Optimal pruning level** for GLU-MLP layers
2. **Performance-Carbon emission trade-offs** at different expansion ratios


---

**Powered by [OptiPFair](https://github.com/peremartra/optipfair)** - Structured Pruning for GLU Architectures

If this research helps your work:
- ⭐ Star [the repo](https://github.com/peremartra/optipfair)
- 📖 Read the [documentation](https://peremartra.github.io/optipfair/)
- 🐛 Report issues or suggest features

---